<a href="https://colab.research.google.com/github/ganaya-nih/NHLBI_Workshop_BigData092023/blob/main/Case_Study_Sleep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digital biomarker use scenario

You have not been sleeping at regular times and you do not feel refreshed after each night’s sleep. You are going through some stressful episode in your work. You suspect that this situation may have to do with your mental well-being, and you decide to track your sleeping schedule using a wearable device and potentially plan a long-term intervention plan.
Altered circadian rhythms are commonly reported among individuals with several psychiatric disorders, including major depressive disorder, bipolar disorder, anxiety and schizophrenia.


# Dataset Background

Demographics:
Total: 66
Age: 20 ~ 47
Gender: 15 female

Wearable data collected using a Garmin Vivosmart HR over a course of 3/6 weeks of normal circadian rhythm, and then 6 weeks of shift work. However, sleep tracking is still a challenge using consumer devices. This could result in some sleep periods that are abnormal to the human circadian rhythm being completely missed. See [this paper](https://pubmed.ncbi.nlm.nih.gov/35416084/) for more information. For the normal circadian rhythm population, 89.7% of the defined main sleep events were recorded. In the cohort that experienced circadian disruptions, only 49.7% of the defined main sleep events were recorded.

The subjects in this sample dataset went through a period of in-class training and then were assigned to start shift work. S1 has day-shift, while S2, S3, S4, and S5 have night-shift. Some sleep periods during the night-shift days were not captured for S2, S3, S4, and S5.

# Download data and load libraries

In [1]:
!gdown 1EX0qGy2_kCVhTEkMjf38ZiFrgteQjcwe
!gdown 1Pdw_ckxBMnTmLxvzRdbKaSj_CT8nbTqv
!gdown 1bAPZvGfFzSCan4YLOber9K0ltva4goRX
!unzip data.zip
!ls

Downloading...
From: https://drive.google.com/uc?id=1EX0qGy2_kCVhTEkMjf38ZiFrgteQjcwe
To: /content/preprocessing_garmin.py
100% 13.7k/13.7k [00:00<00:00, 14.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Pdw_ckxBMnTmLxvzRdbKaSj_CT8nbTqv
To: /content/plotting_sleep.py
100% 3.15k/3.15k [00:00<00:00, 12.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bAPZvGfFzSCan4YLOber9K0ltva4goRX
To: /content/data.zip
100% 21.3M/21.3M [00:00<00:00, 38.4MB/s]
Archive:  data.zip
   creating: data/
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
   creating: data/sleep_cleaned/
   creating: data/aggregate/
   creating: data/epochs_cleaned/
   creating: data/hr_cleaned/
  inflating: data/sleep_cleaned/sleep_cleaned_S1.csv  
  inflating: __MACOSX/data/sleep_cleaned/._sleep_cleaned_S1.csv  
  inflating: data/sleep_cleaned/sleep_cleaned_S2.csv  
  inflating: __MACOSX/data/sleep_cleaned/._sleep_cleaned_S2.csv  
  inflating: data/sleep_cleaned/sleep_cleane

In [2]:
import numpy as np
import pandas as pd
import sklearn
import plotly.graph_objects as go
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.linear_model import LassoCV
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter

# preprocessing_garmin.py and plotting_sleep.py should have been downed above
# importing the necessary helper functions
from preprocessing_garmin import aggregate_by_subject, read_aggregate, \
                                    preprocess_CRP, aggregate_feature_transform, \
                                    extract_Xy
from plotting_sleep import plot_sleep_2D

# suppress pandas warning
pd.set_option('mode.chained_assignment', None)

# provide information on when the shift work started for the subjects
sid_shift_start_info = {"S1": datetime.date(2017, 3, 21),
                        "S2": datetime.date(2017, 9, 25),
                        "S3": datetime.date(2017, 9, 25),
                        "S4": datetime.date(2018, 2, 28),
                        "S5": datetime.date(2018, 2, 28),
                        }


# Data Exploration: sleep 2D plot for a circadian aligned subject

Let's plot the subject's detected sleep periods by Garmin to gain an understanding of the need for sleep imputation.

In [3]:
sid = "S1" # specify a subject
data_dir = './data/' # specify the data directory

# aggregate sleep data, epoch data and heart rate data
# computation will skip if aggregate file has already been computed
aggregate_by_subject(sid, data_dir)
aggregate_df = read_aggregate(sid, data_dir)

# clean-up (preprocess) the aggregate dataframe
summary_aggregate_df = preprocess_CRP(aggregate_df, original=True)
garmin_sleep_df = summary_aggregate_df.loc[summary_aggregate_df["sleepLabel"] == 1, [
    "Date", "Time_In_Day"]]
# plot garmin recorded sleep periods in 2D
plot_sleep_2D(garmin_sleep_df)

This looks pretty consistent, and all sleep periods were tracked by garmin. This person has a regular sleep schedule. Let's look at someone with a irregular sleep schedule.

# Data Exploration: sleep 2D plot for a circadian **misaligned** subject

In [4]:
sid = "S4"
# same computation as above
start_shift_date = sid_shift_start_info[sid]
aggregate_by_subject(sid, data_dir)
aggregate_df = read_aggregate(sid, data_dir)
summary_aggregate_df = preprocess_CRP(aggregate_df, original=True)
garmin_sleep_df = summary_aggregate_df.loc[summary_aggregate_df["sleepLabel"] == 1, [
    "Date", "Time_In_Day"]]
plot_sleep_2D(garmin_sleep_df, plot_schedule_change=True, start_shift_date=start_shift_date)

# Sleep imputation: train a model and plot the results

It seems like Garmin didn't do a very good job picking up the sleep labels, even though we know that this subject wears this wearable device almost always. Now let's build a simple sleep imputation method using logistic regression to impute these information.

We will use a simple logistic regression model to learn the physiological characteristics of sleep for this particular subject using heart rate, heart rate variability and activity intensity measures, using sleep/wake labels that we have confidence about (for example, when the labels are confirmed with their class/training schedules).

In [5]:
# ignore convergence warning for logistic regression models
simplefilter("ignore", category=ConvergenceWarning)

# extract the heart rates, heart rate variability, and activity intensity features
# for every 15 minute epoch, since Garmin reports came in 15 minutes
# for details please see the functions downloaded
features, garmin_sleep_labels = extract_Xy(sid, data_dir)

# defining the pipeline of machine learning algorithms to be used for learning
# physiological characteristics of sleep for this subject
clf = Pipeline([('scaler', StandardScaler()),
               ('classifier', LogisticRegressionCV(penalty='l1', solver='saga'))])
# fit subject data and print out the classification report
clf.fit(features, garmin_sleep_labels)
pred_sleep_labels = clf.predict(features)
print("Individualized model: ")
print(classification_report(garmin_sleep_labels, pred_sleep_labels))

Individualized model: 
              precision    recall  f1-score   support

           0       0.97      0.94      0.95      1515
           1       0.94      0.97      0.96      1618

    accuracy                           0.96      3133
   macro avg       0.96      0.96      0.96      3133
weighted avg       0.96      0.96      0.96      3133



We achieve really good classification result with a logistic regression model and very few features. Now we can use what we learned to impute sleep labels.

In [6]:
# make a prediction on the entire aggregate dataframe for this subject
# features are similarly extracted for each epoch as above
# for details please check out the function downloaded
aggre_fe = aggregate_feature_transform(aggregate_df)
y_pred = clf.predict(aggre_fe)

aggregate_df['predictedSleep'] = y_pred
small_CRP = preprocess_CRP(aggregate_df)

# plot the predicted labels and the garmin provided labels at the same time
garmin_sleep_df = small_CRP.loc[small_CRP["sleepLabel"] == 1, [
    "Date", "Time_In_Day"]]
pred_sleep_df = small_CRP.loc[small_CRP["predictedSleep"] == 1, [
    "Date", "Time_In_Day"]]

plot_sleep_2D(garmin_sleep_df,
            plot_pred = True, plot_schedule_change = True,
            pred_sleep_df = pred_sleep_df, start_shift_date = start_shift_date)


Of course, you will notice that there are some clear mis-classification/mis-imputed values, as evidenced by the sporadic red marks indicating sleep labels. Some of them could be periods of rest or naps during the day. But it is simple to compute and find out the longest consecutive period of sleep, which is not covered in this tutorial.

# Visualization of another circadian **misaligned** subject

In [7]:
sid = "S5"

# plot garmin reported sleep
start_shift_date = sid_shift_start_info[sid]
aggregate_by_subject(sid, data_dir)
aggregate_df = read_aggregate(sid, data_dir)
summary_aggregate_df = preprocess_CRP(aggregate_df, original=True)
garmin_sleep_df = summary_aggregate_df.loc[summary_aggregate_df["sleepLabel"] == 1, [
    "Date", "Time_In_Day"]]
plot_sleep_2D(garmin_sleep_df, plot_schedule_change=True, start_shift_date=start_shift_date)

In [8]:
# train a model
aggre_fe = aggregate_feature_transform(aggregate_df)
features, garmin_sleep_labels = extract_Xy(sid, data_dir)
clf = Pipeline([('scaler', StandardScaler()),
               ('classifier', LogisticRegressionCV(penalty='l1', solver='saga'))])
clf.fit(features, garmin_sleep_labels)

# predict sleep labels
y_pred = clf.predict(aggre_fe)
aggregate_df['predictedSleep'] = y_pred
small_CRP = preprocess_CRP(aggregate_df)

# plot imputed sleep
garmin_sleep_df = small_CRP.loc[small_CRP["sleepLabel"] == 1, [
    "Date", "Time_In_Day"]]
pred_sleep_df = small_CRP.loc[small_CRP["predictedSleep"] == 1, [
    "Date", "Time_In_Day"]]
plot_sleep_2D(garmin_sleep_df,
            plot_pred = True, plot_schedule_change = True,
            pred_sleep_df = pred_sleep_df, start_shift_date = start_shift_date)